# ARM Team Project — Data Preparation & Enrichment (Basel-Land)

## Goal of this notebook
This notebook prepares and enriches the master dataset (municipality-year panel) created from Kanton Basel-Landschaft GeoView API layers.

**Outputs produced by this notebook:**
1. A hypothesis-focused dataset with cleaned and readable column names
2. A cross-sectional EDA dataset for a single selected year (1 row per municipality)
3. (Optional) a panel dataset for later modeling (regression / random forest)

## Research question (working version)
How can average real estate prices of municipalities be modeled using municipal location indicators in Canton Basel-Land?

## Hypothesis (working version)
Municipality-level real estate price indicators are associated with:
- tax rate (Steuerfuss)
- demographic structure (e.g., foreigners share)
- household structure (e.g., avg household size)
- wealth proxy (e.g., income per taxpayer)


## Libraries and settings

In [1]:
# Load Python libraries
import os
import pandas as pd
import numpy as np


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())


# Display settings
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

print("Working directory:", os.getcwd())


/workspaces/ARM_Gruppe-5/02 - Data Preparation
Working directory: /workspaces/ARM_Gruppe-5/02 - Data Preparation


## Import data ( master dataset)


We load the master table produced from the BL GeoView API layers.
This dataset is expected to be a municipality-year panel (keys: `bfs_nummer` + `jahr`).

In [2]:
## Read data
DATA_PATH = "/workspaces/ARM_Gruppe-5/01 - Data Collection/ARM_Master_Table_final.csv"

df_raw = pd.read_csv(DATA_PATH, sep=",", encoding="utf-8")

print("Loaded df_raw with shape:", df_raw.shape)
df_raw.head(5)


Loaded df_raw with shape: (1056, 49)


,jahr,bfs_nummer,gemeinde,ds10630_avg_haushaltsgrosse,ds10080_anteil_auslaender_pct,ds10080_anteil_schweizer_pct,ds10060_1_person,ds10060_2_personen,ds10060_3_personen,ds10060_4_personen,ds10060_5_personen,ds10060_6_oder_mehr_personen,ds10080_ausland,ds10080_schweiz,ds10180_anzahl,ds10200_falle,ds10200_flache_in_m2,ds10200_quadratmeterpreis_chf,ds10230_neu_erstellte_wohnungen,ds10580_steuerfuss_ertrag_juristische_personen_prozent,ds10580_steuerfuss_kapital_juristische_personen_prozent,ds10580_steuerfuss_natuerliche_personen_prozent,ds10580_steuersatz_ertrag_juristische_personen_prozent,ds10580_steuersatz_kapital_juristische_personen_promille,ds10630_anzahl_steuerpflichtige,ds10630_steuerbares_einkommen_chf,ds10680_anfangsbestand,ds10680_geburten,ds10680_todesfaelle,ds10680_zuzuege,ds10680_wegzuege,ds10680_wanderungssaldo,ds10680_endbestand,ds12070_falle,ds12070_flache_in_m2,ds12070_quadratmeterpreis_chf,ds12880_anzahl_bewilligungen,ds12900_anzahl_bauprojekte,ds13010_1_2,ds13010_3,ds13010_4,ds13010_5plus,ds13010_total,ds13030_andere_situation_unbekannt,ds13030_genossenschafter_innen,ds13030_hauseigentuemer_innen,ds13030_mieter_innen_oder_untermieter_innen,ds13030_stockwerk_wohnungseigentuemer_innen,ds13030_total
0,2010,2761,Aesch,0.0000,23.0100,76.9900,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,"2,353.0000","7,874.0000",0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,40894,87,60,732,739,85,40914,0.0000,0.0000,0.0000,74.0000,71.0000,915.9241,"1,329.4116","1,644.7773","2,135.5780","1,419.9320",127.2046,43.2917,"1,461.6278","2,366.0702",484.3123,"4,482.5066"
1,2011,2761,Aesch,0.0000,23.0700,76.9300,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,"2,375.0000","7,920.0000",0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,41149,95,69,789,758,105,41206,0.0000,0.0000,0.0000,71.0000,77.0000,915.9241,"1,329.4116","1,644.7773","2,135.5780","1,419.9320",127.2046,43.2917,"1,461.6278","2,366.0702",484.3123,"4,482.5066"
2,2012,2761,Aesch,2.2700,23.1000,76.9000,"1,317.0000","1,685.0000",642.0000,590.0000,152.0000,56.0000,"2,385.0000","7,939.0000",0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,41313,82,73,743,791,124,41273,0.0000,0.0000,0.0000,58.0000,54.0000,915.9241,"1,329.4116","1,644.7773","2,135.5780","1,419.9320",127.2046,43.2917,"1,461.6278","2,366.0702",484.3123,"4,482.5066"
3,2013,2761,Aesch,2.2700,23.3700,76.6300,"1,320.0000","1,659.0000",653.0000,584.0000,157.0000,56.0000,"2,403.0000","7,879.0000",64.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,41130,89,86,770,776,106,41128,0.0000,0.0000,0.0000,49.0000,49.0000,915.9241,"1,329.4116","1,644.7773","2,135.5780","1,419.9320",127.2046,43.2917,"1,461.6278","2,366.0702",484.3123,"4,482.5066"
4,2014,2761,Aesch,2.2700,23.7500,76.2500,"1,308.0000","1,641.0000",696.0000,549.0000,157.0000,55.0000,"2,439.0000","7,831.0000",80.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,41100,70,85,678,701,61,41062,0.0000,0.0000,0.0000,43.0000,49.0000,915.9241,"1,329.4116","1,644.7773","2,135.5780","1,419.9320",127.2046,43.2917,"1,461.6278","2,366.0702",484.3123,"4,482.5066"


## First inspection & structure check

Goal:
- Understand the structure of the dataset (rows/columns, column names, dtypes)
- Confirm that the key variables for a municipality-year panel exist (`bfs_nummer`, `jahr`, `gemeinde`)
- Get a quick feel for missing values and obvious anomalies

Output:
- Basic dataset overview (`shape`, `columns`, `info`)
- Quick missing-value summary for the key variables


In [3]:
# Basic overview
print("Shape:", df_raw.shape)
print("\nColumns:")
print(list(df_raw.columns))

print("\nInfo:")
df_raw.info()

# Quick check: do key columns exist?
key_cols = ["bfs_nummer", "jahr", "gemeinde"]
missing_keys = [c for c in key_cols if c not in df_raw.columns]
print("\nMissing key columns:", missing_keys if missing_keys else "None")

# Quick missingness for keys
if not missing_keys:
    print("\nMissing values in keys:")
    print(df_raw[key_cols].isna().sum())

# Quick look at year coverage
if "jahr" in df_raw.columns:
    years = df_raw["jahr"].dropna().unique()
    print("\nYear range:", int(np.min(years)), "to", int(np.max(years)))
    print("Number of unique years:", len(years))


Shape: (1056, 49)

Columns:
['jahr', 'bfs_nummer', 'gemeinde', 'ds10630_avg_haushaltsgrosse', 'ds10080_anteil_auslaender_pct', 'ds10080_anteil_schweizer_pct', 'ds10060_1_person', 'ds10060_2_personen', 'ds10060_3_personen', 'ds10060_4_personen', 'ds10060_5_personen', 'ds10060_6_oder_mehr_personen', 'ds10080_ausland', 'ds10080_schweiz', 'ds10180_anzahl', 'ds10200_falle', 'ds10200_flache_in_m2', 'ds10200_quadratmeterpreis_chf', 'ds10230_neu_erstellte_wohnungen', 'ds10580_steuerfuss_ertrag_juristische_personen_prozent', 'ds10580_steuerfuss_kapital_juristische_personen_prozent', 'ds10580_steuerfuss_natuerliche_personen_prozent', 'ds10580_steuersatz_ertrag_juristische_personen_prozent', 'ds10580_steuersatz_kapital_juristische_personen_promille', 'ds10630_anzahl_steuerpflichtige', 'ds10630_steuerbares_einkommen_chf', 'ds10680_anfangsbestand', 'ds10680_geburten', 'ds10680_todesfaelle', 'ds10680_zuzuege', 'ds10680_wegzuege', 'ds10680_wanderungssaldo', 'ds10680_endbestand', 'ds12070_falle', 'ds1

## Panel integrity checks (uniqueness + coverage)

Goal:
- Confirm that each municipality-year combination is unique (no duplicates)
- Confirm that each municipality has the expected number of years (2010–2025 = 16 years)
- Confirm that the dataset contains the expected number of municipalities

Output:
- Duplicate count for the key (`bfs_nummer`, `jahr`)
- Summary table: years per municipality + year min/max
- Count of unique municipalities


In [4]:
# 1) Check duplicates for the municipality-year key
dup_count = df_raw.duplicated(subset=["bfs_nummer", "jahr"]).sum()
print("Duplicate rows for (bfs_nummer, jahr):", int(dup_count))

# If duplicates exist, show them
if dup_count > 0:
    dups = df_raw[df_raw.duplicated(subset=["bfs_nummer", "jahr"], keep=False)].sort_values(["bfs_nummer", "jahr"])
    print("\nDuplicate records (showing first 20):")
    display(dups.head(20))

# 2) Municipality coverage: how many years per municipality?
coverage = (
    df_raw.groupby(["bfs_nummer", "gemeinde"])["jahr"]
    .agg(n_years="nunique", year_min="min", year_max="max")
    .reset_index()
    .sort_values(["n_years", "gemeinde"], ascending=[True, True])
)

print("\nUnique municipalities:", coverage.shape[0])
display(coverage)

# 3) Quick check: expected years per municipality
expected_years = df_raw["jahr"].nunique()
print("\nExpected number of unique years in dataset:", int(expected_years))

not_full = coverage[coverage["n_years"] != expected_years]
print("Municipalities with incomplete year coverage:", not_full.shape[0])

if not_full.shape[0] > 0:
    display(not_full)


Duplicate rows for (bfs_nummer, jahr): 0

Unique municipalities: 66


,bfs_nummer,gemeinde,n_years,year_min,year_max
0,2761,Aesch,16,2010,2025
1,2762,Allschwil,16,2010,2025
39,2841,Anwil,16,2010,2025
53,2881,Arboldswil,16,2010,2025
26,2821,Arisdorf,16,2010,2025
2,2763,Arlesheim,16,2010,2025
27,2822,Augst,16,2010,2025
54,2882,Bennwil,16,2010,2025
3,2764,Biel-Benken,16,2010,2025
4,2765,Binningen,16,2010,2025



Expected number of unique years in dataset: 16
Municipalities with incomplete year coverage: 0


## Variable dictionary (readable English names with spaces)

Goal:
- Create a variable dictionary mapping:
  raw column name → Clean Name (English, readable, WITH SPACES)
  + label/unit/group/role


Decisions based on dataset definitions:
- ds10200: Residential plot price (Wohnbauland) → KEEP as price indicator
- ds12070: General plot price (Bauland) → IGNORE (not needed)
- ds10180: Business startups/closures → IGNORE (not needed)
- ds13010_total: Average rent price CHF → KEEP

Output:
- A dictionary dataframe we can extend and later use to:
  (1) rename columns for EDA/modeling
  (2) select only hypothesis-relevant variables


In [5]:
# Updated variable dictionary with readable English names (WITH SPACES)

var_dict = [
    # identifiers
    {"rawCol": "jahr", "cleanName": "Year", "unit": "year", "group": "ID", "role": "Identifier"},
    {"rawCol": "bfs_nummer", "cleanName": "BFS ID", "unit": "-", "group": "ID", "role": "Identifier"},
    {"rawCol": "gemeinde", "cleanName": "Municipality", "unit": "-", "group": "ID", "role": "Identifier"},

    # targets / price indicators
    {"rawCol": "ds10200_quadratmeterpreis_chf", "cleanName": "Residential Plot Price CHF per m2",
     "unit": "CHF/m2", "group": "Price", "role": "Target Candidate"},

    # rent indicator (target candidate)
    {"rawCol": "ds13010_total", "cleanName": "Average Rent CHF",
     "unit": "CHF/m2", "group": "Price", "role": "Target Candidate"},

    # predictors aligned with hypothesis
    {"rawCol": "ds10580_steuerfuss_natuerliche_personen_prozent", "cleanName": "Tax Rate Natural Persons %",
     "unit": "%", "group": "Tax", "role": "Predictor"},

    {"rawCol": "ds10080_anteil_auslaender_pct", "cleanName": "Foreigners Share %",
     "unit": "%", "group": "Demographics", "role": "Predictor"},

    {"rawCol": "ds10630_avg_haushaltsgrosse", "cleanName": "Average Household Size",
     "unit": "persons", "group": "Household", "role": "Predictor"},

    # wealth proxy components (we will later derive income per taxpayer)
    {"rawCol": "ds10630_anzahl_steuerpflichtige", "cleanName": "Number of Taxpayers",
     "unit": "count", "group": "Wealth", "role": "Predictor Component"},

    {"rawCol": "ds10630_steuerbares_einkommen_chf", "cleanName": "Taxable Income Total CHF",
     "unit": "CHF", "group": "Wealth", "role": "Predictor Component"},

    # explicitly ignored columns (documented)
    {"rawCol": "ds10180_anzahl", "cleanName": "Business Startups/Closures (Ignored)",
     "unit": "count", "group": "Other", "role": "Ignore"},

    {"rawCol": "ds12070_quadratmeterpreis_chf", "cleanName": "General Plot Price CHF per m2 (Ignored)",
     "unit": "CHF/m2", "group": "Price", "role": "Ignore"},
]

dfVarDict = pd.DataFrame(var_dict)

display(dfVarDict)

# Quick check: dictionary raw columns missing from dataset
missingFromDf = dfVarDict.loc[~dfVarDict["rawCol"].isin(df_raw.columns), "rawCol"].tolist()
print("Dictionary columns missing from df_raw:", missingFromDf if missingFromDf else "None")


,rawCol,cleanName,unit,group,role
0,jahr,Year,year,ID,Identifier
1,bfs_nummer,BFS ID,-,ID,Identifier
2,gemeinde,Municipality,-,ID,Identifier
3,ds10200_quadratmeterpreis_chf,Residential Plot Price CHF per m2,CHF/m2,Price,Target Candidate
4,ds13010_total,Average Rent CHF,CHF/m2,Price,Target Candidate
5,ds10580_steuerfuss_natuerliche_personen_prozent,Tax Rate Natural Persons %,%,Tax,Predictor
6,ds10080_anteil_auslaender_pct,Foreigners Share %,%,Demographics,Predictor
7,ds10630_avg_haushaltsgrosse,Average Household Size,persons,Household,Predictor
8,ds10630_anzahl_steuerpflichtige,Number of Taxpayers,count,Wealth,Predictor Component
9,ds10630_steuerbares_einkommen_chf,Taxable Income Total CHF,CHF,Wealth,Predictor Component


Dictionary columns missing from df_raw: None


## Rename + remove ignored + filter to hypothesis-only columns

Goal:
- Create a rename map from the variable dictionary (rawCol → cleanName)
- Rename columns in a copy of the dataset (keep df_raw unchanged)
- Remove indicators marked as "Ignore"
- Reduce the dataframe to ONLY the variables listed in the dictionary (minus ignored)

Output:
- dfNamed: dataframe with readable English column names (with spaces)
- dfHypSelected: reduced hypothesis-focused dataframe (only selected columns)


In [6]:
# 1) Build rename map from dictionary
renameMap = dict(zip(dfVarDict["rawCol"], dfVarDict["cleanName"]))

# 2) Rename columns (keep df_raw unchanged)
dfNamed = df_raw.copy().rename(columns=renameMap)

print("dfNamed shape:", dfNamed.shape)

# 3) Identify ignored columns (by clean name)
ignoreCleanNames = dfVarDict.loc[dfVarDict["role"].str.lower() == "ignore", "cleanName"].tolist()

# 4) Keep ONLY dictionary columns that are not ignored
keepCleanNames = dfVarDict.loc[dfVarDict["role"].str.lower() != "ignore", "cleanName"].tolist()

# Safety: keep only those columns that exist (prevents errors if dictionary changes later)
keepCleanNames = [c for c in keepCleanNames if c in dfNamed.columns]
ignoreCleanNames = [c for c in ignoreCleanNames if c in dfNamed.columns]

dfHypSelected = dfNamed[keepCleanNames].copy()

print("Ignored columns (dropped):", ignoreCleanNames)
print("Columns kept (hypothesis-focused):", len(keepCleanNames))
print("dfHypSelected shape:", dfHypSelected.shape)

display(dfHypSelected.head())



dfNamed shape: (1056, 49)
Ignored columns (dropped): ['Business Startups/Closures (Ignored)', 'General Plot Price CHF per m2 (Ignored)']
Columns kept (hypothesis-focused): 10
dfHypSelected shape: (1056, 10)


,Year,BFS ID,Municipality,Residential Plot Price CHF per m2,Average Rent CHF,Tax Rate Natural Persons %,Foreigners Share %,Average Household Size,Number of Taxpayers,Taxable Income Total CHF
0,2010,2761,Aesch,0.0000,"1,419.9320",0.0000,23.0100,0.0000,0.0000,0.0000
1,2011,2761,Aesch,0.0000,"1,419.9320",0.0000,23.0700,0.0000,0.0000,0.0000
2,2012,2761,Aesch,0.0000,"1,419.9320",0.0000,23.1000,2.2700,0.0000,0.0000
3,2013,2761,Aesch,0.0000,"1,419.9320",0.0000,23.3700,2.2700,0.0000,0.0000
4,2014,2761,Aesch,0.0000,"1,419.9320",0.0000,23.7500,2.2700,0.0000,0.0000


## Create Wohlstand proxy (Income per Taxpayer)

Goal:
- Operationalize "Wohlstand" as a clean indicator:
  **Income per Taxpayer (CHF)** = Taxable Income Total CHF / Number of Taxpayers
- Keep the original components as well (useful for checks)

Output:
- dfHypEnriched: hypothesis-focused dataset with an additional wealth proxy column
- Quick preview and basic sanity checks for the new variable


In [7]:
dfHypEnriched = dfHypSelected.copy()

# Ensure numeric (safe conversion)
for c in ["Taxable Income Total CHF", "Number of Taxpayers"]:
    if c in dfHypEnriched.columns:
        dfHypEnriched[c] = pd.to_numeric(dfHypEnriched[c], errors="coerce")

# Income per taxpayer (avoid division by zero)
dfHypEnriched["Income per Taxpayer CHF"] = np.where(
    dfHypEnriched["Number of Taxpayers"] > 0,
    dfHypEnriched["Taxable Income Total CHF"] / dfHypEnriched["Number of Taxpayers"],
    np.nan
)

print("dfHypEnriched shape:", dfHypEnriched.shape)
display(dfHypEnriched.head())

print("\nSanity check (Income per Taxpayer CHF):")
display(dfHypEnriched["Income per Taxpayer CHF"].describe())


dfHypEnriched shape: (1056, 11)


,Year,BFS ID,Municipality,Residential Plot Price CHF per m2,Average Rent CHF,Tax Rate Natural Persons %,Foreigners Share %,Average Household Size,Number of Taxpayers,Taxable Income Total CHF,Income per Taxpayer CHF
0,2010,2761,Aesch,0.0000,"1,419.9320",0.0000,23.0100,0.0000,0.0000,0.0000,NaN
1,2011,2761,Aesch,0.0000,"1,419.9320",0.0000,23.0700,0.0000,0.0000,0.0000,NaN
2,2012,2761,Aesch,0.0000,"1,419.9320",0.0000,23.1000,2.2700,0.0000,0.0000,NaN
3,2013,2761,Aesch,0.0000,"1,419.9320",0.0000,23.3700,2.2700,0.0000,0.0000,NaN
4,2014,2761,Aesch,0.0000,"1,419.9320",0.0000,23.7500,2.2700,0.0000,0.0000,NaN



Sanity check (Income per Taxpayer CHF):


count       600.0000
mean     64,082.4152
std      12,642.2364
min      37,048.6211
25%      55,665.2972
50%      62,199.0194
75%      69,366.0069
max     118,429.5547
Name: Income per Taxpayer CHF, dtype: float64

## Year completeness diagnostic (using zeros as “possible missing”)

Goal:
- We want a single cross-sectional EDA year (one row per municipality).
- Many BL API variables may encode "not available" as 0.
- Before doing the real missing-value cleaning, we create a diagnostic:
  For each year, count how many entries are zero across key EDA variables.

Output:
- A year-level table showing:
  - number of municipalities in that year
  - total zero count across selected variables
  - zero rate (zeros / total cells)
- A ranked suggestion of the best year to use for cross-sectional EDA


In [8]:
# Variables we care about for cross-sectional EDA (hypothesis)
# (These are the ones where 0 is suspicious and often means "not available".)
edaCoreCols = [
    "Residential Plot Price CHF per m2",
    "Average Rent CHF",
    "Tax Rate Natural Persons %",
    "Foreigners Share %",
    "Average Household Size",
    "Taxable Income Total CHF",
    "Number of Taxpayers",
    "Income per Taxpayer CHF",
]

# Keep only those that exist
edaCoreCols = [c for c in edaCoreCols if c in dfHypEnriched.columns]

# Build diagnostics per year
rows = []
for y, sub in dfHypEnriched.groupby("Year"):
    n_muni = sub["Municipality"].nunique()

    # count zeros across core columns (ignoring NaNs)
    zero_cells = (sub[edaCoreCols] == 0).sum().sum()
    total_cells = sub[edaCoreCols].shape[0] * sub[edaCoreCols].shape[1]
    zero_rate = zero_cells / total_cells if total_cells > 0 else np.nan

    rows.append({
        "Year": int(y),
        "Municipalities": int(n_muni),
        "ZeroCellsInCoreVars": int(zero_cells),
        "TotalCoreCells": int(total_cells),
        "ZeroRate": float(zero_rate),
    })

dfYearDiag = pd.DataFrame(rows).sort_values(["ZeroRate", "ZeroCellsInCoreVars", "Year"]).reset_index(drop=True)

display(dfYearDiag)
print("\nTop 5 best years by lowest ZeroRate:")
display(dfYearDiag.head(5))


,Year,Municipalities,ZeroCellsInCoreVars,TotalCoreCells,ZeroRate
0,2013,66,55,528,0.1042
1,2017,66,56,528,0.1061
2,2019,66,58,528,0.1098
3,2016,66,59,528,0.1117
4,2018,66,60,528,0.1136
5,2014,66,61,528,0.1155
6,2015,66,66,528,0.1250
7,2021,66,120,528,0.2273
8,2020,66,124,528,0.2348
9,2022,66,124,528,0.2348



Top 5 best years by lowest ZeroRate:


,Year,Municipalities,ZeroCellsInCoreVars,TotalCoreCells,ZeroRate
0,2013,66,55,528,0.1042
1,2017,66,56,528,0.1061
2,2019,66,58,528,0.1098
3,2016,66,59,528,0.1117
4,2018,66,60,528,0.1136


## Set the EDA year (fixed to 2019)

Goal:
- For cross-sectional EDA (one row per municipality), we fix the analysis year to **2019**.
- We still keep the year diagnostic table (Cell 11) for transparency and justification.

Output:
- EDA_YEAR = 2019
- Quick preview of the dataset for 2019 (rows and municipalities)


In [9]:
EDA_YEAR = 2019
print("Selected EDA_YEAR:", EDA_YEAR)

df2019 = dfHypEnriched[dfHypEnriched["Year"] == EDA_YEAR].copy()

print("Rows in selected year:", df2019.shape[0])
print("Unique municipalities in selected year:", df2019["Municipality"].nunique())

display(df2019.sort_values(["Municipality"]).head(66))


Selected EDA_YEAR: 2019
Rows in selected year: 66
Unique municipalities in selected year: 66


,Year,BFS ID,Municipality,Residential Plot Price CHF per m2,Average Rent CHF,Tax Rate Natural Persons %,Foreigners Share %,Average Household Size,Number of Taxpayers,Taxable Income Total CHF,Income per Taxpayer CHF
9,2019,2761,Aesch,0.0000,"1,417.6280",0.0000,25.3600,2.2000,0.0000,0.0000,NaN
25,2019,2762,Allschwil,"1,901.0000","1,386.3910",58.0000,27.5400,2.0900,"13,177.0000","913,135,532.0000","69,297.6802"
41,2019,2841,Anwil,0.0000,"1,273.5419",63.0000,6.0600,2.4400,339.0000,"18,955,513.0000","55,915.9676"
57,2019,2881,Arboldswil,517.0000,"1,297.8560",60.0000,6.4100,2.4100,358.0000,"25,765,667.0000","71,971.1369"
73,2019,2821,Arisdorf,769.0000,"1,492.7620",59.0000,11.5100,2.4200,"1,019.0000","71,871,032.0000","70,530.9441"
89,2019,2763,Arlesheim,"1,848.0000","1,483.1575",45.0000,20.0400,2.1300,"5,898.0000","516,429,749.0000","87,560.1473"
105,2019,2822,Augst,0.0000,"1,397.6237",53.0000,23.4100,1.9500,667.0000,"43,974,449.0000","65,928.7091"
121,2019,2882,Bennwil,0.0000,"1,218.8683",64.0000,6.9100,2.5100,375.0000,"24,253,476.0000","64,675.9360"
137,2019,2764,Biel-Benken,"1,571.0000","1,698.3284",46.0000,14.7500,2.4100,"1,985.0000","191,287,231.0000","96,366.3632"
153,2019,2765,Binningen,"2,337.0000","1,483.7724",48.0000,27.2300,2.1100,"9,849.0000","976,468,268.0000","99,143.8997"


## Create the cross-sectional EDA dataset (one row per municipality)

Goal:
- Use the already filtered dataframe `df2019` (Year = 2019)
- Ensure we have exactly one row per municipality (cross-sectional dataset)
- Sort the dataset for readability and confirm integrity (no duplicates)

Output:
- dfEda2019: 2019 cross-sectional dataset (expected: 9 rows)
- Duplicate checks for Municipality and BFS ID


## Focus only on rental prices (drop residential plot price)

Goal:
- Remove the residential plot price indicator because it has many missing values.
- From now on, the main target variable is:
  **Average Rent CHF**

Output:
- dfEdaYearRent: cross-sectional EDA dataset without the residential plot price column


In [10]:
# Create cross-sectional EDA dataset for 2019
dfEda2019 = df2019.copy()

# Sort for readability
dfEda2019 = dfEda2019.sort_values(["Municipality"]).reset_index(drop=True)

print("dfEda2019 shape:", dfEda2019.shape)

# Check duplicates at municipality level (should be 0)
dup_muni = dfEda2019.duplicated(subset=["Municipality"]).sum()
dup_bfs = dfEda2019.duplicated(subset=["BFS ID"]).sum()

print("Duplicate Municipality rows:", int(dup_muni))
print("Duplicate BFS ID rows:", int(dup_bfs))

display(dfEda2019)


dfEda2019 shape: (66, 11)
Duplicate Municipality rows: 0
Duplicate BFS ID rows: 0


,Year,BFS ID,Municipality,Residential Plot Price CHF per m2,Average Rent CHF,Tax Rate Natural Persons %,Foreigners Share %,Average Household Size,Number of Taxpayers,Taxable Income Total CHF,Income per Taxpayer CHF
0,2019,2761,Aesch,0.0000,"1,417.6280",0.0000,25.3600,2.2000,0.0000,0.0000,NaN
1,2019,2762,Allschwil,"1,901.0000","1,386.3910",58.0000,27.5400,2.0900,"13,177.0000","913,135,532.0000","69,297.6802"
2,2019,2841,Anwil,0.0000,"1,273.5419",63.0000,6.0600,2.4400,339.0000,"18,955,513.0000","55,915.9676"
3,2019,2881,Arboldswil,517.0000,"1,297.8560",60.0000,6.4100,2.4100,358.0000,"25,765,667.0000","71,971.1369"
4,2019,2821,Arisdorf,769.0000,"1,492.7620",59.0000,11.5100,2.4200,"1,019.0000","71,871,032.0000","70,530.9441"
5,2019,2763,Arlesheim,"1,848.0000","1,483.1575",45.0000,20.0400,2.1300,"5,898.0000","516,429,749.0000","87,560.1473"
6,2019,2822,Augst,0.0000,"1,397.6237",53.0000,23.4100,1.9500,667.0000,"43,974,449.0000","65,928.7091"
7,2019,2882,Bennwil,0.0000,"1,218.8683",64.0000,6.9100,2.5100,375.0000,"24,253,476.0000","64,675.9360"
8,2019,2764,Biel-Benken,"1,571.0000","1,698.3284",46.0000,14.7500,2.4100,"1,985.0000","191,287,231.0000","96,366.3632"
9,2019,2765,Binningen,"2,337.0000","1,483.7724",48.0000,27.2300,2.1100,"9,849.0000","976,468,268.0000","99,143.8997"


## Focus only on rental prices (drop residential plot price)

Goal:
- Remove the column "Residential Plot Price CHF per m2" because it contains many missing values.
- From now on, we treat rental prices as the main target variable:
  **Average Rent CHF**

Output:
- dfEda2019Rent: same as dfEda2019 but without residential plot price column


In [11]:
dfEda2019Rent = dfEda2019.copy()

col_to_drop = "Residential Plot Price CHF per m2"
if col_to_drop in dfEda2019Rent.columns:
    dfEda2019Rent = dfEda2019Rent.drop(columns=[col_to_drop])
    print(f"Dropped column: {col_to_drop}")
else:
    print(f"Column not found (already removed): {col_to_drop}")

print("dfEda2019Rent shape:", dfEda2019Rent.shape)
display(dfEda2019Rent)


Dropped column: Residential Plot Price CHF per m2
dfEda2019Rent shape: (66, 10)


,Year,BFS ID,Municipality,Average Rent CHF,Tax Rate Natural Persons %,Foreigners Share %,Average Household Size,Number of Taxpayers,Taxable Income Total CHF,Income per Taxpayer CHF
0,2019,2761,Aesch,"1,417.6280",0.0000,25.3600,2.2000,0.0000,0.0000,NaN
1,2019,2762,Allschwil,"1,386.3910",58.0000,27.5400,2.0900,"13,177.0000","913,135,532.0000","69,297.6802"
2,2019,2841,Anwil,"1,273.5419",63.0000,6.0600,2.4400,339.0000,"18,955,513.0000","55,915.9676"
3,2019,2881,Arboldswil,"1,297.8560",60.0000,6.4100,2.4100,358.0000,"25,765,667.0000","71,971.1369"
4,2019,2821,Arisdorf,"1,492.7620",59.0000,11.5100,2.4200,"1,019.0000","71,871,032.0000","70,530.9441"
5,2019,2763,Arlesheim,"1,483.1575",45.0000,20.0400,2.1300,"5,898.0000","516,429,749.0000","87,560.1473"
6,2019,2822,Augst,"1,397.6237",53.0000,23.4100,1.9500,667.0000,"43,974,449.0000","65,928.7091"
7,2019,2882,Bennwil,"1,218.8683",64.0000,6.9100,2.5100,375.0000,"24,253,476.0000","64,675.9360"
8,2019,2764,Biel-Benken,"1,698.3284",46.0000,14.7500,2.4100,"1,985.0000","191,287,231.0000","96,366.3632"
9,2019,2765,Binningen,"1,483.7724",48.0000,27.2300,2.1100,"9,849.0000","976,468,268.0000","99,143.8997"


## Remove missing values (treat rent = 0 as missing) and count complete municipalities

Goal:
- Create a complete-case dataset for EDA by removing municipalities that have missing values
  in the required EDA core variables.
- Additionally, treat **Average Rent CHF per m2 == 0** as missing (not a real rent value).

Output:
- dfEda2019Complete: dataset with only municipalities that have complete data
- Count of municipalities remaining
- List of removed municipalities (if any)


In [12]:
# Define the core variables required for EDA (rent target + key predictors + wealth proxy)
core_cols = [
    "Year",
    "BFS ID",
    "Municipality",
    "Average Rent CHF",
    "Tax Rate Natural Persons %",
    "Foreigners Share %",
    "Average Household Size",
    "Income per Taxpayer CHF",
]

# Keep only columns that exist (safety)
core_cols = [c for c in core_cols if c in dfEda2019Rent.columns]

dfCore2019 = dfEda2019Rent[core_cols].copy()

# Treat rent == 0 as missing
rent_col = "Average Rent CHF"
if rent_col in dfCore2019.columns:
    dfCore2019.loc[dfCore2019[rent_col] == 0, rent_col] = np.nan

# Track municipalities before dropping
muni_before = set(dfCore2019["Municipality"].unique())

# Drop rows with ANY missing values across core columns
dfEda2019Complete = dfCore2019.dropna(axis=0, how="any").copy()

muni_after = set(dfEda2019Complete["Municipality"].unique())
removed = sorted(list(muni_before - muni_after))

print("Complete-case dataset shape:", dfEda2019Complete.shape)
print("Municipalities with complete data:", len(muni_after), "out of", len(muni_before))

if removed:
    print("\nMunicipalities removed due to missing values (including rent==0 treated as missing):")
    for m in removed:
        print("-", m)
else:
    print("\nNo municipalities removed (all complete).")

display(dfEda2019Complete.sort_values(["Municipality"]).head(30))


Complete-case dataset shape: (58, 8)
Municipalities with complete data: 58 out of 66

Municipalities removed due to missing values (including rent==0 treated as missing):
- Aesch
- Burg im Leimental
- Häfelfingen
- Kilchberg
- Lampenberg
- Oberdorf
- Reinach
- Rickenbach


,Year,BFS ID,Municipality,Average Rent CHF,Tax Rate Natural Persons %,Foreigners Share %,Average Household Size,Income per Taxpayer CHF
1,2019,2762,Allschwil,"1,386.3910",58.0000,27.5400,2.0900,"69,297.6802"
2,2019,2841,Anwil,"1,273.5419",63.0000,6.0600,2.4400,"55,915.9676"
3,2019,2881,Arboldswil,"1,297.8560",60.0000,6.4100,2.4100,"71,971.1369"
4,2019,2821,Arisdorf,"1,492.7620",59.0000,11.5100,2.4200,"70,530.9441"
5,2019,2763,Arlesheim,"1,483.1575",45.0000,20.0400,2.1300,"87,560.1473"
6,2019,2822,Augst,"1,397.6237",53.0000,23.4100,1.9500,"65,928.7091"
7,2019,2882,Bennwil,"1,218.8683",64.0000,6.9100,2.5100,"64,675.9360"
8,2019,2764,Biel-Benken,"1,698.3284",46.0000,14.7500,2.4100,"96,366.3632"
9,2019,2765,Binningen,"1,483.7724",48.0000,27.2300,2.1100,"99,143.8997"
10,2019,2766,Birsfelden,"1,240.3278",62.0000,30.6800,1.9600,"55,344.9286"


## Export final cross-sectional EDA dataset (2019)

Goal:
- Save the complete-case 2019 dataset as the official input for the EDA notebook.

Output:
- A CSV file exported to disk


In [13]:
OUTPUT_PATH = f"/workspaces/ARM_Gruppe-5/02 - Data Preparation/Data/eda_dataset_{EDA_YEAR}_complete.csv"

dfEda2019Complete.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")

print("Saved:", OUTPUT_PATH)
print("Saved shape:", dfEda2019Complete.shape)


Saved: /workspaces/ARM_Gruppe-5/02 - Data Preparation/Data/eda_dataset_2019_complete.csv
Saved shape: (58, 8)


### Jupyter notebook --footer info-- 

In [14]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2026-01-10 23:27:22
Python Version: 3.11.14
-----------------------------------
